In [1]:
##Representation Learning for Attributed Multiplex Heterogeneous Network.

In [2]:
import numpy as np
import pdb
import tqdm
from walk import RWGraph
import networkx as nx
from collections import defaultdict
from gensim.models.keyedvectors import Vocab
from numpy import random
from six import iteritems
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import torch.optim as optim
from sklearn.metrics import (auc, f1_score, precision_recall_curve, roc_auc_score)
from scipy.stats import truncnorm
import torch.nn.functional as F

In [3]:
#路径设定
file_name = '../Tdata'
#超参数设定
embedding_size = 200
embedding_u_size = 30
dim_a = 20 #attention维度
att_head = 1
neighbor_samples = 10
schema = None # metapath的制定schema,此处没有指定
num_walks = 20 #路径数
walk_length = 10 #
window_size = 8 #上下文窗口大小,应小于walk_length

In [4]:
#数据读取
def load_training_data(f_name):
    print('We are loading data from:', f_name)
    edge_data_by_type = dict()
    all_edges = list()
    all_nodes = list()
    with open(f_name, 'r') as f:
        for line in f:
            words = line[:-1].split(' ')
            if words[0] not in edge_data_by_type:
                edge_data_by_type[words[0]] = list()
            x, y = words[1], words[2]
            edge_data_by_type[words[0]].append((x, y))
            all_edges.append((x, y))
            all_nodes.append(x)
            all_nodes.append(y)
    all_nodes = list(set(all_nodes))
    all_edges = list(set(all_edges))
    edge_data_by_type['Base'] = all_edges
    print('Total training nodes: ' + str(len(all_nodes)))
    return edge_data_by_type
def load_testing_data(f_name):
    print('We are loading data from:', f_name)
    true_edge_data_by_type = dict()
    false_edge_data_by_type = dict()
    all_edges = list()
    all_nodes = list()
    with open(f_name, 'r') as f:
        for line in f:
            words = line[:-1].split(' ')
            x, y = words[1], words[2]
            if int(words[3]) == 1:
                if words[0] not in true_edge_data_by_type:
                    true_edge_data_by_type[words[0]] = list()
                true_edge_data_by_type[words[0]].append((x, y))
            else:
                if words[0] not in false_edge_data_by_type:
                    false_edge_data_by_type[words[0]] = list()
                false_edge_data_by_type[words[0]].append((x, y))
            all_nodes.append(x)
            all_nodes.append(y)
    all_nodes = list(set(all_nodes))
    return true_edge_data_by_type, false_edge_data_by_type

In [5]:
network_data = load_training_data(file_name + '/train.txt')
valid_true_data_by_edge, valid_false_data_by_edge = load_testing_data(file_name + '/valid.txt')
testing_true_data_by_edge, testing_false_data_by_edge = load_testing_data(file_name + '/test.txt')

We are loading data from: ../Tdata/train.txt
Total training nodes: 511
We are loading data from: ../Tdata/valid.txt
We are loading data from: ../Tdata/test.txt


In [6]:
#路径生成
#从edgse生成Graph
def get_G_from_edges(edges):
    edge_dict = dict()
    for edge in edges:
        edge_key = str(edge[0]) + '_' + str(edge[1])
        if edge_key not in edge_dict:
            edge_dict[edge_key] = 1
        else:
            edge_dict[edge_key] += 1
    tmp_G = nx.Graph()
    for edge_key in edge_dict:
        weight = edge_dict[edge_key]
        x = edge_key.split('_')[0]
        y = edge_key.split('_')[1]
        tmp_G.add_edge(x, y)
        tmp_G[x][y]['weight'] = weight
    return tmp_G

def generate_walks(network_data):
    base_network = network_data['Base']
    #if schema is not None:
    #    node_type = load_node_type('../Tdata/node_type.txt')
    #else:
    #    node_type = None
    node_type = None
        
    base_walker = RWGraph(get_G_from_edges(base_network), node_type=node_type)
    base_walks = base_walker.simulate_walks(num_walks, walk_length, schema=None)
    all_walks = []
    for layer_id in network_data:
        if layer_id == 'Base':
            continue

        tmp_data = network_data[layer_id]
        # start to do the random walk on a layer

        layer_walker = RWGraph(get_G_from_edges(tmp_data))
        layer_walks = layer_walker.simulate_walks(num_walks, walk_length)

        all_walks.append(layer_walks)

    print('路径生成完毕！')

    return base_walks, all_walks

In [7]:
base_walks, all_walks = generate_walks(network_data)

路径生成完毕！


In [8]:
#生成index2word
def generate_vocab(all_walks):
    index2word = []
    raw_vocab = defaultdict(int)

    for walks in all_walks:
        for walk in walks:
            for word in walk:
                raw_vocab[word] += 1

    vocab = {}
    for word, v in iteritems(raw_vocab):
        vocab[word] = Vocab(count=v, index=len(index2word))
        index2word.append(word)

    index2word.sort(key=lambda word: vocab[word].count, reverse=True)
    for i, word in enumerate(index2word):
        vocab[word].index = i
    
    return vocab, index2word

In [9]:
vocab, index2word = generate_vocab([base_walks])


In [10]:
feature_dic = {}
with open(file_name+'/feature.txt', 'r') as f:
    first = True
    for line in f:
        if first:
            first = False
            continue
        items = line.strip().split()
        feature_dic[items[0]] = items[1:]
if feature_dic is not None:
    feature_dim = len(list(feature_dic.values())[0])
    print('feature dimension: ' + str(feature_dim))
    features = np.zeros((len(vocab), feature_dim), dtype=np.float32)
    for key, value in feature_dic.items():
        if key in vocab:
            features[vocab[key].index, :] = np.array(value)
feature_dic = torch.Tensor(features)

feature dimension: 142


In [11]:
def generate_pairs(all_walks, vocab):
    pairs = []
    skip_window = window_size // 2
    for layer_id, walks in enumerate(all_walks):
        for walk in walks:
            for i in range(len(walk)):
                for j in range(1, skip_window + 1):
                    if i - j >= 0:
                        pairs.append((vocab[walk[i]].index, vocab[walk[i - j]].index, layer_id))
                    if i + j < len(walk):
                        pairs.append((vocab[walk[i]].index, vocab[walk[i + j]].index, layer_id))
    return pairs

In [12]:
train_pairs = generate_pairs(all_walks, vocab)

In [13]:
# 确保Base边在最后一个
edge_types = list(network_data.keys())
if edge_types[-1] != 'Base':
    edge_types.sort()
    edge_types.remove('Base')
    edge_types.append('Base')

In [14]:
# 网络参数计算
num_nodes = len(index2word)
edge_type_count = len(edge_types) - 1
u_num = edge_type_count
neighbors = [[[] for __ in range(edge_type_count)] for _ in range(num_nodes)]

In [15]:
#根据neighbor_samples去截断、填充邻居，如果一个实体点没有邻居，则认为它的邻居是他自己
for r in range(edge_type_count):
    g = network_data[edge_types[r]]
    for (x, y) in g:
        ix = vocab[x].index
        iy = vocab[y].index
        neighbors[ix][r].append(iy)
        neighbors[iy][r].append(ix)
    for i in range(num_nodes):
        if len(neighbors[i][r]) == 0:
            neighbors[i][r] = [i] * neighbor_samples
        elif len(neighbors[i][r]) < neighbor_samples:
            neighbors[i][r].extend(list(np.random.choice(neighbors[i][r], size=neighbor_samples-len(neighbors[i][r]))))
        elif len(neighbors[i][r]) > neighbor_samples:
            neighbors[i][r] = list(np.random.choice(neighbors[i][r], size=neighbor_samples))

In [16]:
# 进行负采样
def negative_sampling(targets, unigram_table, k):
    batch_size = targets.size(0)
    neg_samples = []
    for i in range(batch_size):
        nsample = []
        target_index = targets[i].item()
        while len(nsample) < k:
            neg = random.choice(unigram_table)
            if neg == target_index:
                continue
            nsample.append(neg)
        neg_samples.append(np.array(nsample)) 
    return np.array(neg_samples)

In [17]:
def evaluate(model, true_edges, false_edges):

    true_list = list()
    prediction_list = list()
    true_num = 0
    for edge in true_edges:
        tmp_score = get_score(model, str(edge[0]), str(edge[1]))
        
        if tmp_score is not None:
            true_list.append(1)
            prediction_list.append(tmp_score)
            true_num += 1
    for edge in false_edges:
        tmp_score = get_score(model, str(edge[0]), str(edge[1]))
        
        if tmp_score is not None:
            true_list.append(0)
            prediction_list.append(tmp_score)

    sorted_pred = prediction_list[:]
    sorted_pred.sort()
    threshold = sorted_pred[-true_num]
    y_pred = np.zeros(len(prediction_list), dtype=np.int32)
    for i in range(len(prediction_list)):
        if prediction_list[i] >= threshold:
            y_pred[i] = 1
    y_true = np.array(true_list)
    y_scores = np.array(prediction_list)
    ps, rs, _ = precision_recall_curve(y_true, y_scores)
    return roc_auc_score(y_true, y_scores), f1_score(y_true, y_pred), auc(rs, ps)

In [18]:
def get_score(local_model, node1, node2):

    try:
        vector1 = np.array(local_model[node1].view(-1).tolist())
        vector2 = np.array(local_model[node2].view(-1).tolist())
        return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    except Exception as e:
        pass

In [19]:
def getBatch(pairs, neighbors, batch_size):
    n_batches = (len(pairs) + (batch_size - 1)) // batch_size
    for idx in range(n_batches):
        x, y, t, neigh = [], [], [], []
        for i in range(batch_size):
            index = idx * batch_size + i
            if index >= len(pairs):
                break
            x.append(pairs[index][0])
            y.append(pairs[index][1])
            t.append(pairs[index][2])
            neigh.append(neighbors[pairs[index][0]])
        yield (np.array(x).astype(np.int32), np.array(y).reshape(-1, 1).astype(np.int32), np.array(t).astype(np.int32), np.array(neigh).astype(np.int32)) 

In [20]:
def truncated_normal_(tensor, mean=0, std=1):
    size = tensor.shape
    tmp = tensor.new_empty(size + (4,)).normal_()
    valid = (tmp < 2) & (tmp > -2)
    ind = valid.max(-1, keepdim=True)[1]
    tensor.data.copy_(tmp.gather(-1, ind).squeeze(-1))
    tensor.data.mul_(std).add_(mean)
    return tensor

In [21]:
# 网络结构,核心代码
class MyLayer(nn.Module):
    def __init__(self, num_nodes, embedding_size, u_num, dim_a, att_head, edge_type_count, neighbor_samples, feature_dic):
        super(MyLayer, self).__init__()
        # attention参数传递
        self.neighbor_samples = neighbor_samples
        self.u_num = u_num
        self.att_head = att_head
        self.embedding_size = embedding_size
        self.feature_dim = feature_dic.shape[1]
        self.edge_type_count = edge_type_count
        self.embedding_u_size = embedding_u_size
        self.feature_dic = feature_dic # Tensor(node_num*feature_dim)
        # 初始化权重
        self.neigh_feature_trans = nn.Parameter(torch.Tensor(edge_type_count, feature_dim, embedding_u_size))
        truncated_normal_(self.neigh_feature_trans,std=1.0 / math.sqrt(embedding_size))
        self.neigh_linear_1 = nn.Parameter(torch.Tensor(edge_type_count, embedding_u_size, dim_a))
        truncated_normal_(self.neigh_linear_1,std=1.0 / math.sqrt(embedding_size))
        self.neigh_linear_2 = nn.Parameter(torch.Tensor(edge_type_count, dim_a, att_head))
        truncated_normal_(self.neigh_linear_2,std=1.0 / math.sqrt(embedding_size))
        self.neigh_linear_last = nn.Parameter(torch.Tensor(edge_type_count, embedding_u_size, embedding_size // att_head))
        truncated_normal_(self.neigh_linear_last,std=1.0 / math.sqrt(embedding_size))
        self.feature_layer1 = nn.Parameter(torch.Tensor(feature_dim, embedding_size))
        truncated_normal_(self.feature_layer1,std=1.0 / math.sqrt(embedding_size))
        self.feature_layer2 = nn.Parameter(torch.Tensor(feature_dim, embedding_size))
        truncated_normal_(self.feature_layer2)
    def forward(self,input_node,neigh_type,node_neigh):
        # input_node:LongTensor(1) 输入实体点
        # node_neigh:LongTensor(edge_type_num*neigh_sample) 输入实体点的邻居
        # neigh_type:LongTensor(1) 实体之间的关系
        batch_size = max(1,input_node.shape[0])
        node_feature = torch.index_select(self.feature_dic, 0, input_node)
        # pdb.set_trace()# 提取输入node的特征
        node_embed = torch.matmul(node_feature,self.feature_layer1)
        node_weight = torch.matmul(node_feature,self.feature_layer2)
        #pdb.set_trace()
        node_neigh = torch.unbind(node_neigh, dim=1)
        neigh_type_embedding = torch.empty([batch_size,self.edge_type_count,self.embedding_u_size]).cuda()
        for j in range(batch_size):
            neigh_feature_temp = torch.cat([torch.matmul(torch.index_select(self.feature_dic, 0, node_neigh[i][j]).reshape(-1, self.feature_dim), self.neigh_feature_trans[i].reshape(self.feature_dim, self.embedding_u_size)) for i in range(self.edge_type_count)],0)           
            neigh_feature_temp = neigh_feature_temp.reshape(self.edge_type_count, -1, self.neighbor_samples, self.embedding_u_size)
            neigh_type_embedding[j] = torch.mean(neigh_feature_temp,2).transpose(1,0)
        # attention层
        # pdb.set_trace()
        trans_w_s1 = torch.index_select(self.neigh_linear_1, 0, neigh_type)
        trans_w_s2 = torch.index_select(self.neigh_linear_2, 0, neigh_type)
        trans_w = torch.index_select(self.neigh_linear_last, 0, neigh_type)
        attention = nn.Tanh()(torch.matmul(neigh_type_embedding,trans_w_s1))
        attention = torch.matmul(attention,trans_w_s2)
        attention = nn.Softmax()(attention.view(-1,self.u_num))
        attention = attention.view(-1, self.att_head, self.u_num)        
        node_type_embed = torch.matmul(attention,neigh_type_embedding)
        node_embed = node_embed + torch.matmul(node_type_embed,trans_w).view(-1,self.embedding_size) + node_weight
        last_node_embed = nn.functional.normalize(node_embed, p=2, dim=1, eps=1e-12, out=None)
        return last_node_embed.reshape(batch_size,1,-1)     

In [22]:
class MyNet(nn.Module):
    def __init__(self,num_nodes, embedding_size, u_num, dim_a, att_head, edge_type_count, feature_dic, neighbor_samples):
        super(MyNet, self).__init__()
        self.Embedding_layer = MyLayer(num_nodes, embedding_size, u_num, dim_a, att_head, edge_type_count, neighbor_samples,feature_dic)  
        self.embedding_u = nn.Embedding(num_nodes, embedding_size)
        self.num_nodes = num_nodes
        self.num_sampled = 5
    def forward(self, input_node, node_neigh, neigh_type, label_node):
        n = input_node.shape[0]
        #pdb.set_trace()
        weights = torch.ones((self.num_nodes,), dtype=torch.float) / (self.num_nodes)                                                              
        #weights[label_node.tolist()] = 0
        negs = torch.multinomial(
            weights, self.num_sampled * n, replacement=True
        ).view(n, self.num_sampled).cuda()
        center_embeds = self.Embedding_layer(input_node,neigh_type,node_neigh)
        log_target = torch.log(torch.sigmoid(torch.sum(torch.bmm(self.embedding_u(label_node),center_embeds.transpose(1, 2)), 1))).squeeze()
        noise = torch.neg(self.embedding_u(negs))
        sum_log_sampled = torch.sum(
            torch.log(torch.sigmoid(torch.bmm(noise, center_embeds.transpose(1, 2)))), 1
        ).squeeze()
        loss = log_target + sum_log_sampled        
        return -loss.sum() / n
    def prediction(self,input_node,neigh_type,node_neigh):
        node_neigh = node_neigh.unsqueeze(0)
        return self.Embedding_layer(input_node,neigh_type,node_neigh)

In [23]:
# 模型训练参数
BATCH_SIZE =  1000
EPOCH = 100
NEG = 3
set_patience = 4

In [24]:
losses = []
model = MyNet(num_nodes, embedding_size, u_num, dim_a, att_head, edge_type_count, feature_dic.cuda(), neighbor_samples).cuda()
optimizer = optim.Adam(model.parameters())
g_iter = 0
best_score = 0
patience = 0

In [25]:
for epoch in range(EPOCH):
    random.shuffle(train_pairs)
    batches = getBatch(train_pairs,neighbors,BATCH_SIZE)
    avg_loss = 0.0
    for i,batch in enumerate(getBatch(train_pairs,neighbors,BATCH_SIZE)):

        input_node = torch.LongTensor(batch[0]).cuda()
        label_node = torch.LongTensor(batch[1]).cuda()
        neigh_type = torch.LongTensor(batch[2]).cuda()
        node_neigh = torch.LongTensor(batch[3]).cuda()
        optimizer.zero_grad()
        loss = model(input_node, node_neigh, neigh_type, label_node)        
        loss.backward()
        optimizer.step()
        avg_loss += loss.data.tolist()
        if i % 100 == 0:
            print("Epoch : %d, mean_loss : %.02f" % (epoch, avg_loss / (i + 1)))
        #if i>2:break

    final_model = dict(zip(edge_types[:-1], [dict() for _ in range(edge_type_count)]))
    for i in range(edge_type_count):
        for j in range(num_nodes):
            input_node = torch.LongTensor([j]).cuda()
            node_neigh = torch.LongTensor(neighbors[j]).cuda()
            neigh_type = torch.LongTensor([i]).cuda()
            # pdb.set_trace()
            final_model[edge_types[i]][index2word[j]] = model.prediction(input_node,neigh_type,node_neigh)
            
    valid_aucs, valid_f1s, valid_prs = [], [], []
    test_aucs, test_f1s, test_prs = [], [], []
    for i in range(edge_type_count):
        # pdb.set_trace()
        tmp_auc, tmp_f1, tmp_pr = evaluate(final_model[edge_types[i]], valid_true_data_by_edge[edge_types[i]], valid_false_data_by_edge[edge_types[i]])
        valid_aucs.append(tmp_auc)
        valid_f1s.append(tmp_f1)
        valid_prs.append(tmp_pr)

        tmp_auc, tmp_f1, tmp_pr = evaluate(final_model[edge_types[i]], testing_true_data_by_edge[edge_types[i]], testing_false_data_by_edge[edge_types[i]])
        test_aucs.append(tmp_auc)
        test_f1s.append(tmp_f1)
        test_prs.append(tmp_pr)
    print('valid auc:', np.mean(valid_aucs))
    print('valid pr:', np.mean(valid_prs))
    print('valid f1:', np.mean(valid_f1s))

    average_auc = np.mean(test_aucs)
    average_f1 = np.mean(test_f1s)
    average_pr = np.mean(test_prs)

    cur_score = np.mean(valid_aucs)
    if cur_score > best_score:
        best_score = cur_score
        patience = 0
    else:
        patience += 1
        if patience > set_patience:
            print('Early Stopping')
            print('Overall ROC-AUC:',str(average_auc))
            print('Overall PR-AUC',str(average_pr) )
            print('Overall F1:',str(average_f1))
            output_name = '../Tdata/result_pytorch.txt'
            f = open(output_name,'w+')
            f.write('Overall ROC-AUC:' + str(average_auc) + '\n')
            f.write('Overall PR-AUC'+ str(average_pr) + '\n')
            f.write('Overall F1:'+ str(average_f1) + '\n')   
            break   

/root/miniconda3/envs/wld_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch : 0, mean_loss : 4.78
Epoch : 0, mean_loss : 3.82
Epoch : 0, mean_loss : 3.23
Epoch : 0, mean_loss : 2.93
Epoch : 0, mean_loss : 2.74
Epoch : 0, mean_loss : 2.61
Epoch : 0, mean_loss : 2.51
Epoch : 0, mean_loss : 2.42
valid auc: 0.8211900052459129
valid pr: 0.7607368396149966
valid f1: 0.7249266862170087


/root/miniconda3/envs/wld_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch : 1, mean_loss : 1.83
Epoch : 1, mean_loss : 1.74
Epoch : 1, mean_loss : 1.72
Epoch : 1, mean_loss : 1.69
Epoch : 1, mean_loss : 1.67
Epoch : 1, mean_loss : 1.65
Epoch : 1, mean_loss : 1.63
Epoch : 1, mean_loss : 1.61
valid auc: 0.8622485831735193
valid pr: 0.8365777664499239
valid f1: 0.7620967741935484


/root/miniconda3/envs/wld_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

Epoch : 8, mean_loss : 1.21
Epoch : 8, mean_loss : 1.21
Epoch : 8, mean_loss : 1.20
Epoch : 8, mean_loss : 1.20
Epoch : 8, mean_loss : 1.20
Epoch : 8, mean_loss : 1.20
Epoch : 8, mean_loss : 1.20
Epoch : 8, mean_loss : 1.20
valid auc: 0.9230831778192483
valid pr: 0.9089563322772238
valid f1: 0.8399560117302054


/root/miniconda3/envs/wld_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch : 9, mean_loss : 1.22
Epoch : 9, mean_loss : 1.19
Epoch : 9, mean_loss : 1.19
Epoch : 9, mean_loss : 1.19
Epoch : 9, mean_loss : 1.19
Epoch : 9, mean_loss : 1.19
Epoch : 9, mean_loss : 1.19
Epoch : 9, mean_loss : 1.19
valid auc: 0.9206793134733964
valid pr: 0.9076269419642229
valid f1: 0.8431818181818183


/root/miniconda3/envs/wld_pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch : 10, mean_loss : 1.22
Epoch : 10, mean_loss : 1.19
Epoch : 10, mean_loss : 1.18
Epoch : 10, mean_loss : 1.18
Epoch : 10, mean_loss : 1.18
Epoch : 10, mean_loss : 1.18
Epoch : 10, mean_loss : 1.18
Epoch : 10, mean_loss : 1.18
valid auc: 0.9234407383837429
valid pr: 0.9089009428858608
valid f1: 0.8431818181818183
Early Stopping
Overall ROC-AUC: 0.9336145414474002
Overall PR-AUC 0.9088397518884685
Overall F1: 0.8619035458923099


In [ ]:
name=input()
print('hello,',name)